# Normalisation des tirages de Flexique
- ajout d'une colonne phono au format BDLexique

In [1]:
import pandas as pd
import pickle,unicodedata,re,glob

### Sélection des tirages à normaliser
- serie pour la taille des tirages

In [2]:
nomRep="/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/"
nomRep="/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/"
#nomRep="/Users/gilles/Box Sync/2015-Data/"
serie="100Ko"
listeTirages=glob.glob(nomRep+"MGC-%s-*-*-Tirage.pkl"%serie)
listeTirages=glob.glob(nomRep+"MGC-*Tirage.pkl")
#listeTirages=glob.glob(nomRep+"MGC-*Adjectifs.pkl")

### Transformation de la notation API de Flexique vers celle de BDLexique
- phonèmes codés sur un caractère => flexique1graphe
- phonèmes codés sur deux caractères => flexique2, bdlexique2 

In [3]:
flexique1graphe=(u"ʁʃŋɲœɥʒɔøØəɛɑ",u"rSNJ9HZO226èa")
flexique2=[u"ɛ̃",u"œ̃",u"ɑ̃",u"ɔ̃"]
bdlexique2=u"êûâô"
flexique2graphe={flex:bdlexique2[nf] for nf,flex in enumerate(flexique2)}
flexiqueIn = unicode(flexique1graphe[0])
flexiqueNum = [ord(char) for char in flexiqueIn]
bdlexiqueOut = unicode(flexique1graphe[1])
neutralise = dict(zip(flexiqueNum, bdlexiqueOut))

In [4]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8'))
        for bigramme in flexique2graphe:
            temp=temp.replace(bigramme,flexique2graphe[bigramme])
        temp.translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        for bigramme in flexique2graphe:
            chaine=chaine.replace(bigramme,flexique2graphe[bigramme])
        result=chaine.translate(table)
    else:
        result=chaine
    return result

### Vérification de la phonotactique des glides du français
- ajout de diérèses dans les séquences mal-formées
- vérification des séquences consonne+glide à la finale

In [5]:
dierese={"j":"ij", "w":"uw","H":"yH","i":"ij","u":"uw","y":"yH"}

In [6]:
def checkFrench(prononciation):
    result=recoder(prononciation)
    m=re.match(ur"^.*([^ieèEaOouy926êôâ])[jwH]$",result)
    if m:
        print "pb avec un glide final", prononciation
    m=re.match(ur"(.*[ptkbdgfsSvzZ][rl])([jwH])(.*)",result)
    if m:
        n=re.search(ur"[ptkbdgfsSvzZ][rl](wa|Hi|wê)",result)
        if not n:
            glide=m.group(2)
            result=m.group(1)+dierese[glide]+m.group(3)
    m=re.match(ur"(.*)([iuy])([ieEaOouy].*)",result)
    if m:
        glide=m.group(2)
        result=m.group(1)+dierese[glide]+m.group(3)
    return result

## Boucle principale
- pour chaque échantillon sélectionner
 - ajouter la colonne *phono* normalisée BDLexique

In [7]:
for nomEchantillon in listeTirages:
    with open(nomEchantillon,"rb") as input:
        tirage=pickle.load(input)
    tirage["phono"]=tirage["forme"].apply(checkFrench)
    with open(nomEchantillon,"wb") as output:
        pickle.dump(tirage, output, pickle.HIGHEST_PROTOCOL)

In [8]:
tirage[tirage["phono"].str.contains(u"Ø")]

,lexeme,case,forme,freq,phono


# TESTS

In [115]:
listeTirages

['/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-00-T100000-F10047-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-01-T100000-F10028-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-02-T100000-F10053-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-03-T100000-F9950-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-04-T100000-F10012-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-05-T100000-F9989-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-06-T100000-F10073-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-07-T100000-F10095-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-08-T100000-F10028-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-100Ko-09-T100000-F9958-Tirage.pkl',
 '/Users/gilles/Box Sync/2015-Data/Samples/Adjectifs/MGC-10Ko-0

In [112]:
nomEchantillon

'/Users/gilles/Box Sync/2015-Data/FlexionAdjectifs/Longitudinal-27-T10000000-F20973-170718-1253.pkl'

In [35]:
tirage1=tirage[tirage["tir1"]>0]

In [36]:
paradigme=pd.pivot_table(tirage1,index="lexeme",values="forme",columns=["case"],aggfunc=sum)
paradigme=paradigme[["ms","mp","fs","fp"]][:]

In [65]:
pFS=paradigme.fs.notnull()
pFP=paradigme.fp.notnull()
pFem=pFS | pFP
pMS=paradigme.ms.notnull()
pMP=paradigme.mp.notnull()
pMsMp=pMS & pMP & ~pFem
pMsFem=pMS & ~pMP & pFem
pMpFem=~pMS & pMP & pFem
pComp=pMS & pMP & pFS & pFP
pFull= pMS & pMP & pFem
print "comp:",len(paradigme[pComp])
print "ms,mp,fs:", len(paradigme[pMS & pMP & pFS & ~pFP])
print "ms,mp,fp:", len(paradigme[pMS & pMP & ~pFS & pFP])
print "ms,fs,fp:", len(paradigme[pMS & ~pMP & pFS & pFP])
print "mp,fs,fp:", len(paradigme[~pMS & pMP & pFS & pFP])
print "ms,mp:", len(paradigme[pMsMp])
print "ms,fs:", len(paradigme[pMS & ~pMP & pFS & ~pFP])
print "ms,fp:", len(paradigme[pMS & ~pMP & ~pFS & pFP])
print "mp,fs:", len(paradigme[~pMS & pMP & pFS & ~pFP])
print "mp,fp:", len(paradigme[~pMS & pMP & ~pFS & pFP])
print "fs,fp:", len(paradigme[~pMS & ~pMP & pFS & pFP])
print "ms:",len(paradigme[pMS & ~pMP & ~pFem ])
print "mp:",len(paradigme[~pMS & pMP & ~pFem ])
print "fs:", len(paradigme[~pMS & ~pMP & pFS & ~pFP])
print "fp:", len(paradigme[~pMS & ~pMP & ~pFS & pFP])
print
print "full:",len(paradigme[pFull])
print "ms,mp:", len(paradigme[pMsMp])
print "ms,fem:", len(paradigme[pMsFem])
print "mp,fem:", len(paradigme[pMpFem])
print "ms:",len(paradigme[pMS & ~pMP & ~pFem ])
print "mp:",len(paradigme[~pMS & pMP & ~pFem ])
print "fem:",len(paradigme[~pMS & ~pMP & pFem ])

comp: 2457
ms,mp,fs: 557
ms,mp,fp: 75
ms,fs,fp: 298
mp,fs,fp: 48
ms,mp: 365
ms,fs: 388
ms,fp: 99
mp,fs: 71
mp,fp: 36
fs,fp: 456
ms: 1174
mp: 452
fs: 710
fp: 318

full: 3089
ms,mp: 365
ms,fem: 785
mp,fem: 155
ms: 1174
mp: 452
fem: 1484
